In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('train_data.csv',
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True, encoding = 'latin1')

In [3]:
df2 = pd.read_csv('test_data.csv',
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True, encoding = 'latin1')

In [4]:
data = df1.append(df2, ignore_index = True)[['asin','categories', 'description', 'review_id','review_text','reviewer','star_rating']]
data = data.rename(columns = {'asin': 'app_id'})

In [5]:
train.append(miss_star_rating).tail(501757-436680)

NameError: name 'train' is not defined

In [7]:
data = data.dropna(submiss_star_ratingset = ['categories'])
data = data[data['categories'].str.contains('game', case = False)]

In [8]:
data = data.dropna(subset = ['review_text'])

In [9]:
miss_star_rating = data[~np.isfinite(data['star_rating'])]
miss_star_rating = miss_star_rating[['review_text', 'star_rating']]
train = data[np.isfinite(data['star_rating'])]

In [ ]:
train = train[['review_text', 'star_rating']]

In [ ]:
text = train['review_text']

ls = text.values.tolist()
import string
exclude = set(string.punctuation)
for i in range(len(ls)):
    ls[i] = ''.join(ch for ch in ls[i] if ch not in exclude)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 0.005,max_df=0.8)
dtm = vectorizer.fit_transform(ls)
terms = vectorizer.get_feature_names()

In [ ]:
X_train = train['star_rating']

In [ ]:
Y = train['star_rating']

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dtm, Y, test_size=0.5, random_state=0)

X_train = pd.DataFrame(X_train.todense(),columns = vectorizer.get_feature_names())

X_test = pd.DataFrame(X_test.todense(),columns = vectorizer.get_feature_names())

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred1 = gnb.predict(X_test)
nb_avg_error = ((abs(y_test - y_pred1)).sum())/len(y_pred1)
print('Naive Bayes Misclassified samples: %d'% (y_test != y_pred1).sum())
print('Naive Bayes average error: %.6f'% nb_avg_error)
from sklearn.metrics import accuracy_score
print('Naive Bayes Accuracy: %.2f'% accuracy_score(y_test, y_pred1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=100, 
                                random_state=1,
                                n_jobs=2)
forest.fit(X_train, y_train)

y_pred_forest = forest.predict(X_test)
rf_avg_error = ((abs(y_test - y_pred_forest)).sum())/len(y_pred_forest)
print('RF Misclassified samples: %d'% (y_test != y_pred_forest).sum())
print('RF average error: %.6f'% rf_avg_error)
from sklearn.metrics import accuracy_score

print('RF Accuracy: %.2f'% accuracy_score(y_test, y_pred_forest))

In [ ]:
from sklearn import linear_model
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

y_train_pred = regr.predict(X_train)
y_test_pred = regr.predict(X_test)
regr_avg_error = ((abs(y_test - y_test_pred)).sum())/len(y_test_pred)
print('Linear regression average error: %.6f'% regr_avg_error)
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

print('MSE train: %.3f, test: %.3f' %(mean_squared_error(y_train, y_train_pred),
                                      mean_squared_error(y_test, y_test_pred)))
 
print('R^2 train: %.3f, test: %.3f' %(r2_score(y_train, y_train_pred),
                                      r2_score(y_test, y_test_pred)))

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1000.0, random_state=0)
lr.fit(X_train, y_train)
        
y_pred = lr.predict(X_test)
print('Misclassified samples: %d' % (y_test != y_pred).sum())

from sklearn.metrics import accuracy_score
lr_avg_error = ((abs(y_test - y_pred)).sum())/len(y_pred)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))    
print('Logistic regression average error: %.6f'% lr_avg_error)

In [ ]:
text = train.append(miss_star_rating)['review_text']

In [ ]:
ls = text.values.tolist()
import string
exclude = set(string.punctuation)
for i in range(len(ls)):
    ls[i] = ''.join(ch for ch in ls[i] if ch not in exclude)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 0.002,max_df=0.8)
dtm = vectorizer.fit_transform(ls)
terms = vectorizer.get_feature_names()

In [ ]:
X = pd.DataFrame(dtm.todense(),columns = vectorizer.get_feature_names())

In [ ]:
X_known = X.head(436680)
Y_known = train.append(miss_star_rating)['star_rating'].head(436680)
X_unknown = X.tail(501757-436680)
Y_unknown = train.append(miss_star_rating)['star_rating'].tail(501757-436680)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1000.0, random_state=0)
lr.fit(X_known, Y_known)

In [ ]:
y_pred = lr.predict(X_unknown)

In [ ]:
miss_star_rating = data[~np.isfinite(data['star_rating'])]
miss_star_rating['star_rating'] = y_pred

In [ ]:
train = data[np.isfinite(data['star_rating'])]

In [ ]:
data

In [ ]:
data = data[['app_id','categories', 'description', 'review_id','review_text','reviewer','star_rating']]

In [ ]:
data.to_csv('data.csv', index = False)